In [ ]:
!pip install ../input/timm-0-1-30/timm-0.1.30-py3-none-any.whl

In [ ]:
import numpy as np 
import pandas as pd 
import os
from PIL import Image, ImageFilter
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import *
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from torchvision import models
import time
from tqdm import tqdm
import random
import timm
import sys
sys.path.append('../input/autoaug')
from auto_augment import AutoAugment, Cutout

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True

In [ ]:
seed_everything(0)

num_classes = 5
bs = 64
lr = 5e-4
IMG_SIZE = 224

In [ ]:
train_path = '../input/cassava-leaf-disease-classification/train_images/'
test_path = '../input/cassava-leaf-disease-classification/test_images/'

train_csv = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
sample = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

In [ ]:
train_csv.head()

In [ ]:
class MyDataset(Dataset):
    
    def __init__(self, dataframe, transform=None, test=False):
        self.df = dataframe
        self.transform = transform
        self.test = test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        label = self.df.label.values[idx]
        p = self.df.image_id.values[idx]
        
        if self.test == False:
            p_path = train_path + p
        else:
            p_path = test_path + p
            
        image = cv2.imread(p_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = transforms.ToPILImage()(image)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    AutoAugment(),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])


testset      = MyDataset(sample, transform=test_transform, test=True)
test_loader  = DataLoader(testset, batch_size=bs, shuffle=False, num_workers=4)

In [ ]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def train_model(model, epoch):
    model.train() 
    
    losses = AverageMeter()
    accs = AverageMeter()
    
    tk = tqdm(train_loader, total=len(train_loader), position=0, leave=True)
    for idx, (imgs, labels) in enumerate(tk):
        imgs_train, labels_train = imgs.cuda(), labels.cuda().long()
        output_train = model(imgs_train)

        loss = criterion(output_train, labels_train)
        
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step() 
        
        accs.update((output_train.argmax(1)==labels_train).sum().item()/imgs_train.size(0),imgs_train.size(0))
        losses.update(loss.item(), imgs_train.size(0))

        tk.set_postfix(loss=losses.avg,acc=accs.avg)
        
    return losses.avg


def test_model(model):    
    model.eval()
    
    losses = AverageMeter()
    accs = AverageMeter()
    
    with torch.no_grad():
        tk = tqdm(val_loader, total=len(val_loader), position=0, leave=True)
        for idx, (imgs, labels) in enumerate(tk):
            imgs_valid, labels_valid = imgs.cuda(), labels.cuda().long()
            output_valid = model(imgs_valid)
            
            loss = criterion(output_valid, labels_valid)

            losses.update(loss.item(), imgs_valid.size(0))
            accs.update((output_valid.argmax(1)==labels_valid).sum().item()/imgs_valid.size(0),imgs_valid.size(0))
            
            tk.set_postfix(loss=losses.avg,acc=accs.avg)

            
    return losses.avg,accs.avg

In [ ]:
# train_df, val_df = train_test_split(train_csv,test_size=0.2,stratify=train_csv.label)

# trainset = MyDataset(train_df, transform=train_transform)
# train_loader = DataLoader(trainset, batch_size=bs, shuffle=True, num_workers=4)

# valset = MyDataset(val_df, transform=test_transform)
# val_loader = DataLoader(valset, batch_size=bs, shuffle=False, num_workers=4)

# model = timm.create_model('tf_efficientnet_b0_ns', pretrained=True, num_classes=num_classes)
# model.cuda()

# optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0)
# criterion = nn.CrossEntropyLoss()
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, \
#                                                        patience=1, verbose=True, min_lr=1e-5)

# best_acc = 0
# n_epochs = 10

# for epoch in range(n_epochs):
#     train_loss = train_model(model, epoch)
#     val_loss, acc = test_model(model)

#     if acc > best_acc:
#         best_acc = acc
#         torch.save(model.state_dict(), 'weight.pt')

#     print('current_val_acc:', acc, 'best_val_acc:', best_acc)

#     scheduler.step(acc)

In [ ]:
model = timm.create_model('tf_efficientnet_b0_ns', pretrained=False, num_classes=num_classes)
model.cuda()

model.load_state_dict(torch.load('../input/cassava-b0/weight.pt'))
model.eval()

In [ ]:
test_pred = []

with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader, position=0, leave=True)):
        images, _ = data
        images = images.cuda()

        pred = model(images)

        pred = pred.argmax(1).cpu().detach().numpy().astype('int')

        test_pred.extend(pred)

sample.label = test_pred
sample.to_csv('submission.csv',index=False)

In [ ]:
sample